# Mapas cloropléticos

Caderno Jupyter referente à **Issue #5**: *Mapa coroplético de origens e destinos de pedestres com filtros e granularidades* 

Mapa(s) coroplético(s) que represente(m) o volume de viagens de pedestres, com as seguintes funcionalidades:
- Filtros censitários: raça, sexo, faixa etária
- Granularidades de regiões: zonas OD, distritos e municipios


Imports

In [95]:
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt

Carregando datasets [OD]

In [96]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
districts = geopandas.read_file(od_districts_shapefile_path)

od_counties_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Municipios_2023.shp" 
counties = geopandas.read_file(od_counties_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Criando um mapa com zoom em São Paulo

In [97]:
# SP coordinates
lat, lon = -23.5918264, -46.6576057

sp_map = folium.Map(location=[lat, lon], zoom_start=10)

sp_map.save("../built_maps/sp_map.html")

Filtra viagens de pedestres do DataFrame da OD

In [98]:
pedestrians = od.query('MODOPRIN == 17') ## 17: pedestrians

Cria um dataframe que contém a representatividade de viagens de pedestres nas zonas OD

**TODO**: criar um dataframe com mesmo objetivo para municípios

In [106]:
# Criar dataframe de origens
origens = pedestrians[["ZONA_O", "FE_VIA"]].rename(columns={"ZONA_O": "ZONA"})

# Criar dataframe de destinos
destinos = pedestrians[["ZONA_D", "FE_VIA"]].rename(columns={"ZONA_D": "ZONA"})

# Concatenar origens e destinos
zonas = pd.concat([origens, destinos], ignore_index=True)

# Agrupar e somar
od_zones_rep = (
    zonas.groupby("ZONA", as_index=False)["FE_VIA"].sum()
    .rename(columns={"FE_VIA": "FE_VIA_TOTAL"})
)

# Calcular percentual (corrigido para viagens contadas 2x)
od_zones_rep["PERC_TOTAL"] = (
    od_zones_rep["FE_VIA_TOTAL"] / (pedestrians["FE_VIA"].sum() * 2) * 100
)

od_zones_rep = od_zones_rep.rename(columns={"ZONA": "NumeroZona"})
od_zones_rep["NumeroZona"] = od_zones_rep["NumeroZona"].astype(int) 

print(od_zones_rep.head())
print(od_zones_rep["PERC_TOTAL"].sum())

   NumeroZona  FE_VIA_TOTAL  PERC_TOTAL
0           1  35469.615143    0.176352
1           2  24969.806514    0.124148
2           3  31158.008191    0.154915
3           4  47402.928819    0.235683
4           5  45706.267246    0.227248
100.0


Muda o CRS de DF de zonas OD

In [101]:
print(od_zones.crs)

EPSG:22523


Altera o CRS de zonas e casta a coluna NumeroZona ppara int

In [108]:
od_zones = od_zones.to_crs("EPSG:4326")
od_zones["NumeroZona"] = od_zones["NumeroZona"].astype(int)

Cria um mapa coroplético de pedestres nas zonas OD em São Paulo

In [110]:
z = folium.Choropleth(
    geo_data=od_zones.to_json(),
    name="choropleth",
    data=od_zones_rep,
    columns=["NumeroZona", "PERC_TOTAL"],
    key_on="feature.properties.NumeroZona",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Viagens de Pedestres (%)",
    nan_fill_color="white",
    tooltip=folium.features.GeoJsonTooltip(
        fields=["NumeroZona", "NomeZona"],
        aliases=["Zona:", "Nome:"],
        localize=True
    )
).add_to(sp_map)

sp_map.save("../built_maps/pedestrians_odzones.html")